In [2]:
import pandas as pd
import numpy as np
import sys, os

from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

p = Path(".")

In [6]:
df = pd.read_csv(p / "../logs/2022_07_13T10_42_log.log")

In [7]:
df

,tick,last,bid,pmid,ask,best_bid,mid,best_ask,bid_vwap,ask_vwap,...,time_factor,vol_spread,trend,trend_confidence,gtrend,gtrend_confidence,position,current_bid,position_vwap,current_ask
0,95,0.00,18.50,18.51,18.52,18.50,18.510,18.52,18.458269,18.522444,...,0.000000,0.00,-0.001273,1.751808,-0.000826,1.575205,0.0,0.00,0.00,0.00
1,97,18.31,18.28,18.31,18.34,18.28,18.310,18.34,18.280000,18.340120,...,0.555328,0.13,-0.001418,1.926713,-0.000891,1.703701,25000.0,18.38,18.38,18.64
2,99,18.29,18.26,18.29,18.32,18.26,18.290,18.32,18.259635,18.320000,...,0.543434,0.13,-0.001346,1.822166,-0.000883,1.708663,5000.0,17.53,18.38,18.33
3,100,18.21,18.23,18.23,18.24,18.23,18.235,18.24,18.177851,18.242291,...,0.535720,0.13,-0.001414,1.901938,-0.000919,1.787695,5000.0,18.03,18.38,18.36
4,100,18.21,18.23,18.23,18.24,18.23,18.235,18.24,18.177851,18.242291,...,0.535720,0.13,-0.001414,1.901938,-0.000919,1.787695,5000.0,18.03,18.38,18.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2454,298,16.10,16.07,16.08,16.09,16.07,16.080,16.09,16.066544,16.133556,...,0.472359,0.11,-0.000376,0.514471,-0.000710,2.209832,0.0,15.84,0.00,16.06
2455,298,16.10,16.07,16.08,16.09,16.07,16.080,16.09,16.066544,16.133556,...,0.472359,0.11,-0.000376,0.514471,-0.000710,2.209832,0.0,15.84,0.00,16.06
2456,298,16.09,16.07,16.08,16.09,16.07,16.080,16.09,16.066544,16.133604,...,0.472359,0.11,-0.000389,0.532193,-0.000712,2.216605,0.0,15.84,0.00,16.06
2457,298,16.09,16.07,16.08,16.09,16.07,16.080,16.09,16.066544,16.133604,...,0.472359,0.11,-0.000226,0.313226,-0.000703,2.196239,0.0,15.84,0.00,16.06
